# Importing and loading necessary libraries

In [1]:
import numpy as np
import cv2
import random
import os

from imutils import paths


from keras.models import Sequential
from keras.layers.core import Activation
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras import applications
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import load_model
from keras import backend as K
from collections import defaultdict

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, precision_recall_fscore_support, average_precision_score, confusion_matrix
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Load and preprocess data

In [2]:
#dataset path

path = 'data/cat_and_dog_1000'
# path = 'data/cat_and_dog_1000'

In [3]:
# Initialise required variables
data = []
labels = []
features = []
RANDOM_STATE = 42

In [4]:
# Initialise required hyperparameters
IMAGE_DIM = 48
TEST_SPLIT = 0.20
BATCH_SIZE = 32
EPOCHS = 3
INIT_LR = 1e-3

In [5]:
# Extracting the class information from the given folder : Class name and corresponding number of files
class_dir_paths = os.listdir(path)
class_dict = {}
idx = 0

for class_dir_path in class_dir_paths:
    if (os.path.isdir(os.path.join(path, class_dir_path))):
        class_name = class_dir_path
        class_dict[class_name] = idx
        idx += 1

print("Class information : ", class_dict)

Class information :  {'cat': 0, 'dog': 1}


In [6]:
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images(path)))
random.seed(RANDOM_STATE)
random.shuffle(imagePaths)

In [25]:
len(imagePaths)

2001

In [26]:
imagePaths[0]

'data/cat_and_dog_1000/cat/cat.813.jpg'

In [12]:
# loop over the input images
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIM, IMAGE_DIM))
    image = img_to_array(image)
    feature = image_to_feature_vector(image)
    data.append(image)
    features.append(feature)

    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(class_dict[label])

print("Total number of images readed, ", len(data))
print("Total number of labels extracted, ", len(labels))

Total number of images readed,  2001
Total number of labels extracted,  2001


# Simple Neural network

In [132]:
len(features[0])

3072

In [13]:
features = np.array(features, dtype="float") / 255.0

(ttrainX, ttestX, ttrainY, ttestY) = train_test_split(
    features, labels, test_size=TEST_SPLIT, random_state=RANDOM_STATE)

# convert the labels from integers to vectors
ttrainY = to_categorical(ttrainY, num_classes=len(class_dict))
ttestY = to_categorical(ttestY, num_classes=len(class_dict))

print("Number of image in training set ", len(ttrainX), len(ttrainY))
print("Number of image in tesing set ", len(ttestX), len(ttestY))

Number of image in training set  1600 1600
Number of image in tesing set  401 401


In [134]:
# define the architecture of the network
model = Sequential()
model.add(Dense(768, input_dim=3072, activation="relu"))
model.add(Dense(384, activation="relu"))
model.add(Dense(2))
model.add(Activation("softmax"))

In [135]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 768)               2360064   
_________________________________________________________________
dense_24 (Dense)             (None, 384)               295296    
_________________________________________________________________
dense_25 (Dense)             (None, 2)                 770       
_________________________________________________________________
activation_13 (Activation)   (None, 2)                 0         
Total params: 2,656,130
Trainable params: 2,656,130
Non-trainable params: 0
_________________________________________________________________


In [136]:
# Compiling the model
sgd = SGD(lr=0.01)
model.compile(loss="binary_crossentropy", optimizer=sgd, metrics=["accuracy"])

In [137]:
print("[INFO] training network...")
nn_history = model.fit(
    ttrainX,
    ttrainY,
    epochs=EPOCHS,
    batch_size=32,
    verbose=1,
    validation_data=(ttestX, ttestY))

[INFO] training network...
Train on 1600 samples, validate on 401 samples
Epoch 1/3
1600/1600 [==============================] - 2s 993us/step - loss: 0.7351 - acc: 0.5225 - val_loss: 0.6828 - val_acc: 0.5586
Epoch 2/3
1600/1600 [==============================] - 1s 632us/step - loss: 0.6839 - acc: 0.5675 - val_loss: 0.6916 - val_acc: 0.5387
Epoch 3/3
1600/1600 [==============================] - 1s 666us/step - loss: 0.6727 - acc: 0.5881 - val_loss: 0.6756 - val_acc: 0.5611


In [138]:
print_score(model, ttrainX, ttrainY, ttestX, ttestY)

401/401 [==============================] - 0s 111us/step
************ Training Metrics ************
Accuracy =  0.546875
Confusion Matrix
[[782   4]
 [721  93]]
************ Testing Metrics ************
Accuracy =  0.5610972573780953
Confusion Matrix
[[209   6]
 [170  16]]


In [ ]:
# plot_save_metrics(nn_history, 'nn_plot')

# Convolutional Neural network

In [14]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [15]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=TEST_SPLIT, random_state=RANDOM_STATE)

print("Number of image in training set ", len(trainX), len(trainY))
print("Number of image in tesing set ", len(testX), len(testY))

Number of image in training set  1600 1600
Number of image in tesing set  401 401


In [16]:
# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=len(class_dict))
testY = to_categorical(testY, num_classes=len(class_dict))

In [17]:
def simple_net(width, height, depth, classes):

    finalAct = "softmax" if classes > 2 else "sigmoid"

    # initialize the model
    model = Sequential()
    inputShape = (height, width, depth)

    # if we are using "channels first", update the input shape
    if K.image_data_format() == "channels_first":
        inputShape = (depth, height, width)

    # first set of CONV => RELU => POOL layers
    model.add(Conv2D(20, (5, 5), padding="same", input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # second set of CONV => RELU => POOL layers
    model.add(Conv2D(50, (5, 5), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # first (and only) set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation("relu"))

    # softmax classifier
    model.add(Dense(classes))
    model.add(Activation(finalAct))

    # return the constructed network architecture
    return model

In [18]:
aug = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest")

In [19]:
model = simple_net(IMAGE_DIM, IMAGE_DIM, 3, len(class_dict))

In [20]:
# Choose the optimizer
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

# Complile the model
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [21]:
print("[INFO] training network...")
H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1)

[INFO] training network...
Epoch 1/3
50/50 [==============================] - 9s 175ms/step - loss: 0.7445 - acc: 0.5109 - val_loss: 0.6913 - val_acc: 0.5112
Epoch 2/3
50/50 [==============================] - 8s 165ms/step - loss: 0.6897 - acc: 0.5406 - val_loss: 0.6795 - val_acc: 0.5411
Epoch 3/3
50/50 [==============================] - 8s 165ms/step - loss: 0.6794 - acc: 0.5741 - val_loss: 0.6947 - val_acc: 0.4838


In [22]:
print_score(model,trainX, trainY, testX, testY)

401/401 [==============================] - 1s 1ms/step
************ Training Metrics ************
Accuracy =  0.5278125
Confusion Matrix
[[ 41 745]
 [ 10 804]]
************ Testing Metrics ************
Accuracy =  0.4837905242853331
Confusion Matrix
[[ 12 203]
 [  3 183]]


## To save the model

In [24]:
model.save('keras_model')

## Load the saved model and do prediction

In [29]:
# load the image
image = cv2.imread('data/cat_and_dog_1000/cat/cat.813.jpg')
orig = image.copy()

# pre-process the image for classification
image = cv2.resize(image, (IMAGE_DIM, IMAGE_DIM))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

# load the trained convolutional neural network
print("[INFO] loading network...")
model = load_model('keras_model')


[INFO] loading network...


In [33]:
(cat, dog) = model.predict(image)[0]

In [36]:
# One with the highest value is our prediction
(cat, dog)

(0.4179721, 0.5898914)

# Transfer Learning

In [148]:
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(IMAGE_DIM, IMAGE_DIM, 3))

In [149]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [150]:
#Freeze all the layers
for layer in base_model.layers:
    layer.trainable = False

In [151]:
# Building model based on Function API
X = base_model.output
X = Flatten()(X)  

#Adding last FC layers based on hyperparams given

X = Dense(128, activation='relu')(X)
X = Dense(128, activation='relu')(X)

# for layer_param in hyperparameters['top_layers']:
# 	X = self.layers[layer_param[0]](layer_param[1], activation=layer_param[2])(X)

#Adding dropout
X = Dropout(0.5)(X)

#Adding the last layer for prediction
predictions = Dense(2, activation='sigmoid')(X)

#creating the final model
model = Model(base_model.input, predictions)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [152]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [153]:
print("[INFO] training network...")
H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1)

[INFO] training network...
Epoch 1/3
50/50 [==============================] - 25s 492ms/step - loss: 0.6457 - acc: 0.6200 - val_loss: 0.5857 - val_acc: 0.6746
Epoch 2/3
50/50 [==============================] - 25s 496ms/step - loss: 0.5952 - acc: 0.6797 - val_loss: 0.5640 - val_acc: 0.7057
Epoch 3/3
50/50 [==============================] - 25s 492ms/step - loss: 0.5774 - acc: 0.6972 - val_loss: 0.5521 - val_acc: 0.7170


In [154]:
print_score(model,trainX, trainY, testX, testY)

401/401 [==============================] - 5s 12ms/step
************ Training Metrics ************
Accuracy =  0.7646875
Confusion Matrix
[[581 205]
 [169 645]]
************ Testing Metrics ************
Accuracy =  0.7169576050931973
Confusion Matrix
[[142  73]
 [ 40 146]]


# Finetuning

In [155]:
# Unfreeze only the last few layers
N_LAYERS_TO_FREEZE = 15

for layer in model.layers[:15]:
    layer.trainable = False

for layer in model.layers[15:]:
    layer.trainable = True
    
# Unfreeze all the layers
# for layer in model.layers:
#     layer.trainable = True

In [157]:
print("[INFO] Finetuning the network...")
model.compile(
    optimizer=SGD(lr=0.001, momentum=0.9),
    loss='binary_crossentropy',
    metrics=['accuracy'])

[INFO] Finetuning the network...


In [158]:
H_tune = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1)

Epoch 1/3
50/50 [==============================] - 45s 891ms/step - loss: 0.5629 - acc: 0.7191 - val_loss: 0.5045 - val_acc: 0.7481
Epoch 2/3
50/50 [==============================] - 43s 869ms/step - loss: 0.5050 - acc: 0.7534 - val_loss: 0.5376 - val_acc: 0.7357
Epoch 3/3
50/50 [==============================] - 45s 892ms/step - loss: 0.4884 - acc: 0.7706 - val_loss: 0.5564 - val_acc: 0.7170


In [160]:
print_score(model,trainX, trainY, testX, testY)

401/401 [==============================] - 5s 12ms/step
************ Training Metrics ************
Accuracy =  0.79625
Confusion Matrix
[[507 279]
 [ 42 772]]
************ Testing Metrics ************
Accuracy =  0.7169576061336774
Confusion Matrix
[[118  97]
 [ 15 171]]


# utils

In [9]:
def plot_save_metrics(model_history):

    # plot the training loss and accuracy
    plt.style.use("ggplot")
    plt.figure()
    N = EPOCHS
    plt.plot(
        np.arange(0, N), model_history.history["loss"], label="train_loss")
    plt.plot(
        np.arange(0, N), model_history.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), model_history.history["acc"], label="train_acc")
    plt.plot(
        np.arange(0, N), model_history.history["val_acc"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    return plt

In [10]:
def print_score(model, trainX, trainY, testX, testY):

    train_score = model.evaluate(trainX, trainY)
    train_predictions = model.predict(trainX)
    train_predictions = np.round(train_predictions)

    test_score = model.evaluate(testX, testY)
    test_predictions = model.predict(testX)
    test_predictions = np.round(test_predictions)

    print("************ Training Metrics ************")

#     print("Loss = ", train_score[0])
    print("Accuracy = ", train_score[1])
#     print("classification Report")
#     print(classification_report(trainY, train_predictions))
    print("Confusion Matrix")
    train_cm = confusion_matrix(
        trainY.argmax(axis=1), train_predictions.argmax(axis=1))
    print(train_cm)

    print("************ Testing Metrics ************")

#     print("Loss = ", test_score[0])
    print("Accuracy = ", test_score[1])
#     print("classification Report")
#     print(classification_report(testY, test_predictions))
    print("Confusion Matrix")
    test_cm = confusion_matrix(
        testY.argmax(axis=1), test_predictions.argmax(axis=1))
    print(test_cm)

In [11]:
# resize the image to a fixed size, then flatten the image into
# a list of raw pixel intensities

def image_to_feature_vector(image, size=(32, 32)):
    return cv2.resize(image, size).flatten()